In [1]:
import numpy as np
import pandas as pd 
import os
from sklearn.model_selection import train_test_split
import PIL
import tensorflow as tf

In [2]:
X = []
y = []

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if len(dirname) <= 60:
            image_directory = dirname + '/' + filename
            image = PIL.Image.open(image_directory).resize((96, 96)).convert('L')
            
            # display(image)
            image_matrix = np.array(image).reshape([96,96,1])/255
            
            X.append(image_matrix)
            y.append(0) if 'n' in filename.lower()[:-3] else y.append(1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
print(f"Size of training set: {len(X_train)}")
print(f"Size of test set: {len(X_test)}")

Size of training set: 189
Size of test set: 64


In [6]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)


In [7]:
def generate_model():
    model = tf.keras.Sequential([
        # first layer (convolutional)
        tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        
        # second layer (convolutional)
        tf.keras.layers.Conv2D(64,kernel_size=(3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(0.25),
        
        # classifier
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

In [8]:
model = generate_model()
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy()])

2022-03-05 13:16:53.425720: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
model.fit(X_train, y_train, epochs=24, batch_size=4, verbose=1)

2022-03-05 13:16:53.598084: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/24
48/48 [==============================] - 5s 74ms/step - loss: 0.6351 - binary_accuracy: 0.6349
Epoch 2/24
48/48 [==============================] - 3s 73ms/step - loss: 0.5904 - binary_accuracy: 0.6984
Epoch 3/24
48/48 [==============================] - 4s 75ms/step - loss: 0.5180 - binary_accuracy: 0.7513
Epoch 4/24
48/48 [==============================] - 4s 75ms/step - loss: 0.4985 - binary_accuracy: 0.7725
Epoch 5/24
48/48 [==============================] - 4s 75ms/step - loss: 0.4015 - binary_accuracy: 0.8307
Epoch 6/24
48/48 [==============================] - 4s 78ms/step - loss: 0.3165 - binary_accuracy: 0.8730
Epoch 7/24
48/48 [==============================] - 4s 75ms/step - loss: 0.2047 - binary_accuracy: 0.9048
Epoch 8/24
48/48 [==============================] - 4s 75ms/step - loss: 0.2067 - binary_accuracy: 0.9206
Epoch 9/24
48/48 [==============================] - 4s 87ms/step - loss: 0.2275 - binary_accuracy: 0.9259
Epoch 10/24
48/48 [===========================

In [10]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

Evaluate on test data
64/64 [==============================] - 1s 11ms/step - loss: 0.6671 - binary_accuracy: 0.8438
test loss, test acc: [0.6671182513237, 0.84375]
